## 다수의 주체가 포함된 뉴스 감정 분석을 위해 모델의 추가적인 Fine-Tuning 수행

- 뉴스 감정 예측 모델의 성능을 향상시키기 위해, **다양한 주체**가 포함된 애매한 문장을 추가적으로 학습하는 Fine-tuning을 진행
- Fine-tuning 이전에는 아래와 같은 문장에서 감정 예측 성능이 부족했으며, 이를 보완하기 위해 학습 데이터에서 **다양한 주체가 포함된 문장**들을 선별하여 Fine-tuning 데이터셋을 생성
- 하지만 선별된 데이터의 양이 적어 학습에 적합하지 않았기 때문에, **K-TACC**를 사용하여 데이터 증강을 진행

---

### Fine-tuning 이전 성능

| 문장                                                                         | 실제 감정 | 예측 감정 |
|----------------------------------------------------------------------------|-------|-------|
| 삼성전자가 실적 발표에서 긍정적인 결과를 보였으나, SK하이닉스는 부진했다...                               | 부정    | 중립    |
| SK하이닉스는 실적 상승을 기록했지만 삼성전자는 다소 실망스러운 실적을 발표했다...                            | 긍정    | 부정    |
| SK하이닉스의 실적 발표 발표에서 클라우드 부문의 성장 둔화가 우려를 불러일으켰다, 아마존는 반도체 부문 시장에서 강세를 보였다... | 부정    | 긍정    |

---

### Fine-tuning 이후 성능

| 문장                                                                         | 실제 감정 | 예측 감정 |
|----------------------------------------------------------------------------|-------|-------|
| 삼성전자가 실적 발표에서 긍정적인 결과를 보였으나, SK하이닉스는 부진했다...                               | 부정    | 부정    |
| SK하이닉스는 실적 상승을 기록했지만 삼성전자는 다소 실망스러운 실적을 발표했다...                            | 긍정    | 긍정    |
| SK하이닉스의 실적 발표 발표에서 클라우드 부문의 성장 둔화가 우려를 불러일으켰다, 아마존는 반도체 부문 시장에서 강세를 보였다... | 부정    | 부정    |

---

* Fine-tuning을 통해 애매한 주체가 포함된 문장에서도 모델의 감정 예측 성능 개선

In [1]:
import pandas as pd
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

---
## 모델과 토크나이저 설정
- KLUE BERT 모델을 금융 뉴스 감정 데이터셋에 맞춰 Fine-Tuning을 수행한 모델 사용

In [2]:
# 모델 경로와 토크나이저 경로 설정
MODEL_PATH = "../sentiment_analysis_model"  # 모델이 저장된 경로
MODEL_NAME = "klue/bert-base"  # 모델 이름

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_PATH)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../sentiment_analysis_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


___
### 데이터 전처리
- 직접 수집한 뉴스데이터에서 다수의 주체가 포함된 문장을 선별하여 생성한 데이터셋 사용
- `description` 열의 형식을 문자열로 변환
- 데이터 셋을 훈련 데이터(80%)와 테스트 데이터(20%) 분리
- 토크나이저를 사용해 **문장 정수 인코딩**, **패딩 및 트리밍 적용**, **문장 벡터화**하여 BERT 모델에 적합한 형태로 변환

In [3]:
# 다양한 주체가 있는 데이터 불러오기
dataset = pd.read_csv("../data/augmented_subject_focus_data.csv", encoding="utf-8", header=None, names=['description', 'sentiment'])

In [6]:
# 데이터 셋 요약 확인
print('데이터 요약')
dataset.info()

# 'description' 컬럼을 문자열 형식으로 변환
dataset['description'] = dataset['description'].astype(str)

# "SK하이닉스", "sk하이닉스" 텍스트 변환
dataset = dataset.applymap(lambda x: x.replace("SK하이닉스", "<SK하이닉스>") if isinstance(x, str) else x)
dataset = dataset.applymap(lambda x: x.replace("sk하이닉스", "<SK하이닉스>") if isinstance(x, str) else x)

# 데이터 확인
print(dataset.head())

# 새로운 토큰 추가
new_tokens = ["<SK하이닉스>"]  # 추가할 새로운 토큰 리스트
tokenizer.add_tokens(new_tokens)

# 토큰 추가에 따라 모델의 임베딩 레이어 크기 재설정
model.resize_token_embeddings(len(tokenizer))

# 데이터 전처리
def encode_data(data, output):
    if output :
        print("데이터 전처리 과정:")
        for i, sentence in enumerate(data['description'].head(3)):  # 상위 3개만 출력
            tokenized = tokenizer.encode_plus(
                sentence,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors="tf"
            )
            print(f"문장 {i+1}: {sentence}")
            print(f"토큰: {tokenizer.tokenize(sentence)}")
            print(f"정수 인코딩: {tokenized['input_ids'][0].numpy()}\n")
        
    # 전체 데이터 토큰화
    return tokenizer(
        data['description'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# 학습 데이터와 테스트 데이터 분리
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# 데이터 전처리
train_encodings = encode_data(train_data, True)
val_encodings = encode_data(val_data, False)

데이터 요약
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  566 non-null    object
 1   sentiment    566 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.0+ KB
                                         description  sentiment
0  삼성전자와 <SK하이닉스>가 각각 152 220 하락한 것을 비롯해 국내 반도체주가...          2
1  이날 삼성전자와 <SK하이닉스>가 저마다 다따로 151 152 220으로 하락한 등...          2
2  삼성전자와 <SK하이닉스>가 각각  152 220 하락한 것을 비롯해 국내 반도체주...          2
3  반면 삼성전자와 SK SK <SK하이닉스>가 이날 각각 152 220 하락한 것을 ...          2
4  전날 삼성전자와 <SK하이닉스>가 각각 152 220으로 하락한 것을 보고 비롯해 ...          2
데이터 전처리 과정:
문장 1: 삼성전자는 기대 이상의 실적을 발표했지만 <SK하이닉스>는 오히려 적기는 하지만 어느 정도로 부진한 결과를 보였다
토큰: ['삼성전자', '##는', '기대', '이상', '##의', '실적', '##을', '발표', '##했', '##지만', '<SK하이닉스>', '는', '오히려', '적기', '##는', '하지만', '어느', '정도', '##로', '부진', '##한', '결과', '##를', '보였', '##다']
정수 인코딩: [    2  4798  2259  3869  

---
### 모델 학습

- 모델 학습 중 성능 향상 및 최적화를 위한 콜백 함수 정의 
    - `EarlyStopping`으로 검증 정확도가 개선되지 않으면 학습 중단
- 데이터 수에 맞춰 적당한 학습률 설정
- 마지막 Epoch 결과 `val_loss: 0.0760`, `val_accuracy: 0.9737`

In [7]:
# 감정 레이블
train_labels = train_data['sentiment'].values
val_labels = val_data['sentiment'].values

# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings), train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), val_labels
))

# 배치 처리 및 데이터 셔플
train_dataset = train_dataset.shuffle(len(train_data)).batch(16)
val_dataset = val_dataset.batch(16)

# 콜백 함수 정의 (EarlyStopping)
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2),
]

# 모델 학습 준비 (Adam 옵티마이저, 손실 함수 설정)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# 모델 학습
model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=callbacks)

# 학습된 모델 저장
model.save_pretrained('../subject_focus_finetuned_model')
tokenizer.save_pretrained("../subject_focus_finetuned_model")

Epoch 1/5
29/29 [==============================] - 36s 521ms/step - loss: 0.7491 - accuracy: 0.7212 - val_loss: 0.4043 - val_accuracy: 0.8772
Epoch 2/5
29/29 [==============================] - 12s 404ms/step - loss: 0.3032 - accuracy: 0.9181 - val_loss: 0.2470 - val_accuracy: 0.9298
Epoch 3/5
29/29 [==============================] - 12s 404ms/step - loss: 0.1921 - accuracy: 0.9580 - val_loss: 0.1506 - val_accuracy: 0.9737
Epoch 4/5
29/29 [==============================] - 12s 402ms/step - loss: 0.1344 - accuracy: 0.9690 - val_loss: 0.1082 - val_accuracy: 0.9737
Epoch 5/5
29/29 [==============================] - 12s 404ms/step - loss: 0.1114 - accuracy: 0.9735 - val_loss: 0.0760 - val_accuracy: 0.9737


('../subject_focus_finetuned_model\\tokenizer_config.json',
 '../subject_focus_finetuned_model\\special_tokens_map.json',
 '../subject_focus_finetuned_model\\vocab.txt',
 '../subject_focus_finetuned_model\\added_tokens.json')